#SMART AI Attendance System Using Face Recognition


SVM=It's a powerful and versatile machine learning algorithm used for both classification and regression tasks.


In [ ]:
#Dataset Creation

import imutils
import time
import cv2
import csv
import os

cascade = 'haarcascade_frontalface_default.xml'
detector = cv2.CascadeClassifier(cascade)


Name = str(input("Enter your Name:"))
Index_No = int(input("Enter Your Index Number:"))
dataset = 'dataset'
sub_data = Name
path = os.path.join(dataset, sub_data)

if not os.path.isdir(path):
  os.mkdir(path)
  print(sub_data)
info = [str(Name), str(Index_No)]
with open('student.csv','a') as csvFile:
    write = csv.writter(csvFile)
    write.writerow(info)
csvFile.close()



print ("Starting Video Stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)
total = 0

while total < 50:
      print(total)
      _,frame = cam.read()
      img =imutils.resize(frame, width =400)
      rects = detector.detectMultiScale(
         cv2.cvtColor(img,cv2.COLOR_BGR2GRAY), SCALEfACTOR=1.1,
         minNeighbors=5, minSize=(30,30))

      for(x, y, w, h) in rects:
        cv2.rectangle(frame, (x,y), (x + w, y +h),(0,255,0),2)
        p = os.path.sep.join([path, "{}.png".format(str(total).zfill(5))])

        cv2.imwrite(p,img)
        total += 1


      cv2.imshow("Frame", frame)
      key = cv2.waitKey(1) & 0xFF
      if key == ord("q"):
        break

cam.release()
cv2.destroyAllWindows()




Enter your Name:fdf
Enter Your Index Number:7567567


FileNotFoundError: ignored

In [ ]:
#Pre-Processing Embeddings


from imutils import paths
import numpy as np
import pickle
import imutils
import cv2
import os

dataset = "dataset"
embeddingFile = "output/embeddings.pickle"
embeddingModel = "openface_nn4.small2.v1.t7"

prototxt = "model/deploy.prototxt"
model= "model/res10_300x300_ssd_iter_140000.caffemodel"

#loading caffe model for face detection

# Detecting face from the Image via Caffe deep learning
detector = cv2.dnn.readNetFromCaffe(prototxt, model)


#loading pytorch model file for extract facial embeddings
#extracting facial embeddings via deep learning feature extraction
embedder = cv2.dnn.readNetFromTorch(embeddingModel)


#getting image paths
imagePaths = list(paths.list_images(dataset))


knownEmbeddings = []
knownNames = []


total = 0
conf = 0.5 #confidence level

#starting reading images one by one to apply face detection and embedding

for (i, imagePath) in enumerate(imagePaths):
  print("Processing image {}/{}".format(i + 1,len(imagePaths)))
  name=imagePath.split(os.path.sep)[-2]
  image = cv2.imread(imagePath)
  image = imutils.resize(image,width=600)
  (h, w) = image.shape[:2]

  #Converting image to blob for dnn face detection
  imageBlob = cv2.dnn.blobFromImage(
      cv2.resize(image, (300,300)),1.0, (300,300),(104.0,177.0,123.0), swapRB= False )

#setting input blob mage
  detector.setInput(imageBlob)

  detections=detector.forward()

  if len(detections) > 0:
    i = np.argmax(detections[0,0, :, 2])
    confidence = detections[ 0, 0, i,2]

    if confidence > conf:
      #ROI range of interest
      box =detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      (startX, startY, endX, endY) = box.astype("int")
      face = image[startY:endY, startX:endX]
      (fH, fW) = face.shape[:2]


      if fW <20 or fH<20:
          continue

      #image to blob for face
      faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96,96), (0, 0, 0), swapRB=False)

      #facial features embedder input image faceblob
      embedder.setinput(faceBlob)
      vec = embedder . forward()
      knownNames.append(name)
      knownEmbeddings.append(vec.flatten())
      total += 1


print("Embedding:{0}".format(total))
data = {"embeddings": knownEmbeddings, "names":knownNames}
f = open(embeddingFile, "wb") #wb= write binary
f.write(pickle.dumps(data))
f.close()
print("Process Completed!")















error: ignored

# Training with Machine Learning

In [ ]:
#using sklearn

from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
import pickle


#Initialization of embedding & recognizer
embeddingFile = "output/embeddings.pickle"
recognizerFile = "output/recognizer.pickle"
labelEncFile = "output/le.pickle"


print("Loading face embeddings...")
data = pickle.loads(open(embeddingFile, "rb").read())


print("Encoding Lables...")
labelEnc = LabelEncoder()
labels = labelEnc.fit_transform(data["names"])



print("Training Model...")
recognizer = SVC(C=1.0, kernel ="linear", probability=True)
recognizer.fit(data["embeddings"], labels)


f = open(recognizerFile, "wb")
f.write(pickle.dumps(recognizer))
f.close()

f=open(labelEncFile, "wb")
f.write(pickle,dumps(LabelEnc))
f.close()

#Recognizing Person

In [1]:
import numpy as np
import imutils
import pickle
import time
import cv2

embeddingModel = "openface.nn4.small2.v1.t7"


enbeddingFile = "output/embeddings.pickle"
recognizerFile = "output/recognizer.pickle"
labelEncFile = "output/le.pickle"
conf = 0.5


print("Loading Face Detector...")
protxt = "model/deploy.protxt"
model = "model/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(prototxt,model)


print("Loading Face Recognizer...")
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

recognizer = pickle.loads(open(recognizerFile, "rb").reada())
le = pickle.loads(open(labelEncFile, "rb").read())


Index_No=""
box = []
print("Starting Video Stream...")
cam = cv2.VideoCapture(1)
time.sleep(2.0)

while True:
    _, frame = cam.read()
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    #parse to the BLOB
    imageBlob = cv2.dnn.blobFromImage(cv2.resize(frame, (300,  300)),1.0, (300, 300),(104.0, 177.0, 123.0), swapRB=False, crop=False)

detector.setInput(imageBlob)
    detections = detector.forward()

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > conf:
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            if fW < 20 or fH < 20:
                continue

            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            text = "{}  : {:.2f}%".format(name, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),(0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()



IndentationError: ignored

#Recognition Person with CSV Database

In [ ]:
from collections import Iterable
import numpy as np
import imutils
import pickle
import time
import cv2
import csv


def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:
            yield item


embeddingFile = "output/embeddings.pickle"
embeddingModel = "openface_nn4.small2.v1.t7"
recognizerFile = "output/recognizer.pickle"
labelEncFile = "output/le.pickle"
conf = 0.5

print("[INFO] loading face detector...")
prototxt = "model/deploy.prototxt"
model = "model/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(prototxt, model)

print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, "rb").read())

Roll_Number = ""
box = []
print("[INFO] starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    _, frame = cam.read()
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    detector.setInput(imageBlob)
    detections = detector.forward()

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > conf:

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            if fW < 20 or fH < 20:
                continue

            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            with open('student.csv', 'r') as csvFile:
                reader = csv.reader(csvFile)
                for row in reader:
                    box = np.append(box, row)
                    name = str(name)
                    if name in row:
                        person = str(row)
                        print(name)
                listString = str(box)
                print(box)
                if name in listString:
                    singleList = list(flatten(box))
                    listlen = len(singleList)
                    Index = singleList.index(name)
                    name = singleList[Index]
                    Roll_Number = singleList[Index + 1]
                    print(Roll_Number)

            text = "{} : {} : {:.2f}%".format(name, Roll_Number, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                          (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()

RecognizingPersonwithCSVDatabase

In [ ]:
from collections import Iterable
import numpy as np
import imutils
import pickle
import time
import cv2
import csv


def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:
            yield item


embeddingFile = "output/embeddings.pickle"
embeddingModel = "openface_nn4.small2.v1.t7"
recognizerFile = "output/recognizer.pickle"
labelEncFile = "output/le.pickle"
conf = 0.5

print("[INFO] loading face detector...")
prototxt = "model/deploy.prototxt"
model = "model/res10_300x300_ssd_iter_140000.caffemodel"
detector = cv2.dnn.readNetFromCaffe(prototxt, model)

print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(embeddingModel)

recognizer = pickle.loads(open(recognizerFile, "rb").read())
le = pickle.loads(open(labelEncFile, "rb").read())

Index_No = ""
box = []
print("[INFO] starting video stream...")
cam = cv2.VideoCapture(0)
time.sleep(2.0)

while True:
    _, frame = cam.read()
    frame = imutils.resize(frame, width=600)
    (h, w) = frame.shape[:2]
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(frame, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    detector.setInput(imageBlob)
    detections = detector.forward()

    for i in range(0, detections.shape[2]):

        confidence = detections[0, 0, i, 2]

        if confidence > conf:

            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            face = frame[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            if fW < 20 or fH < 20:
                continue

            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255, (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            preds = recognizer.predict_proba(vec)[0]
            j = np.argmax(preds)
            proba = preds[j]
            name = le.classes_[j]
            with open('student.csv', 'r') as csvFile:
                reader = csv.reader(csvFile)
                for row in reader:
                    box = np.append(box, row)
                    name = str(name)
                    if name in row:
                        person = str(row)
                        print(name)
                listString = str(box)
                print(box)
                if name in listString:
                    singleList = list(flatten(box))
                    listlen = len(singleList)
                    Index = singleList.index(name)
                    name = singleList[Index]
                    Roll_Number = singleList[Index + 1]
                    print(Roll_Number)

            text = "{} : {} : {:.2f}%".format(name, Roll_Number, proba * 100)
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame, (startX, startY), (endX, endY),
                          (0, 0, 255), 2)
            cv2.putText(frame, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == 27:
        break

cam.release()
cv2.destroyAllWindows()